# Calculation of sensitivities in IBMOS using adjoint techniques

In this document we describe the implementation of the computation of sensitivies in this software. Typically, we formulate an optimization problem where we would like to find the set of parameters $\mu$ that minimizes a given cost functional $J(\mu)$.
Usually, $J(\mu)$ is an expensive function and $\mu$ is a list of a large number of parameters, that may include a combination of the following types of parameters:
 - Shape and/or location of immersed boundaries, directly through the location of grid points, i.e. $\xi_k$ and $\eta_k$, or parameters, e.g. $\xi_k(R,x_0,y_0)$ and $\eta_k(R,x_0,y_0)$ where $R$, $x_0$, $y_0$ are the cylinder radius, and the coordinates of the center.
 - Parameters in the flow equations (mainly Reynolds number).
 - Modification of base flows and/or parameters.
 - Boundary conditions on the immersed boundaries directly though the velocity at the individual lagrangian points, i.e. $u_{Bk}, v_{Bk}$, or parameters, e.g. $u_{Bk}(\omega, \dots)$, $v_{Bk}(\omega, \dots)$.
 - Forcing on the momentum equation and/or divergence-free equation.
 
These parameters can either be constant in time or time dependent, and they can be classified in two categories: (i) parameters that appear in the Jacobian (shape of immersed boundaries, parameters in the equations and baseflow changes) and (ii) parameters that only affect the right hand sides (velocity at the boundaries, and forcing in the momentum and mass conservation equations).

## Steady-state case

The starting point is the optimization problem $$ \min_{x,u} J(x, u) \quad \text{such that} \quad R(x, u) = 0. $$ By introducing the Lagrange multiplier $\lambda$, and the Lagrangian $\mathcal{L}(x, u, \lambda) = J(x,u)-\lambda^T R(x,u)$, we can write the optimization problem as $ \min_{x,u,\lambda} \mathcal{L}(x, u, \lambda)$. 

In thi case, the first-order optimality conditions read then
$$\frac{\delta L}{\delta x} = 0 \longrightarrow \frac{\partial J}{\partial x} - \lambda^T \frac{\partial R}{\partial x} =0$$
$$\frac{\delta L}{\delta u} = 0 \longrightarrow \frac{\partial J}{\partial u} - \lambda^T \frac{\partial R}{\partial u} =0$$
$$\frac{\delta L}{\delta \lambda} = 0 \longrightarrow R(x,u) = 0$$

Here, we focus on the computation of the solution using gradient-based methods, where we use the derivative of the cost functional $$ \frac{\mathrm{d}  J}{\mathrm{d} u} = \frac{\partial J}{\partial x}  \frac{\partial x}{\partial u} + \frac{\partial J}{\partial u},$$
where $\frac{\partial x}{\partial u}$ is the solution of $\frac{\partial R}{\partial x} \frac{\partial x}{\partial u} + \frac{\partial R}{\partial u} = 0$

The computational cost of evaluating $\frac{\mathrm{d}J}{\mathrm{d}u}$ can be reduced by using the adjoint equation:


$$ \frac{\mathrm{d}  J}{\mathrm{d} u} =  \frac{\partial J}{\partial u} + \frac{\partial J}{\partial x} \left(\frac{\partial R}{\partial x}\right)^{-1} \frac{\partial R}{\partial x} \frac{\partial x}{\partial u} =  \frac{\partial J}{\partial u}-\lambda^T \frac{\partial R}{\partial u} $$

# Unsteady case

The optimization problem in this case can be written as
$$ \min J(x_0, x_1, \dots, x_n; u_1, u_2, \dots u_{n}) \quad \text{such that} \quad R(x_0, x_0, x_1; u_1) = 0, \quad R(x_0, x_1, x_2; u_2), \quad \dots \quad R(x_{n-2}, x_{n-1}, x_n; u_n).$$

$$ \min \left(J(x_0, x_1, \dots, x_n; u_1, \dots u_n) -\lambda_{1}^T R(x_0, x_0, x_1; u_1) - \lambda_2^T R(x_0, x_1, x_2; u_2) \dots \lambda_n^T R(x_{n-2}, x_{n-1}, x_n; u_n)\right)$$

## Derivatives wrt Lagrange multipliers...
$$ \partial_{\lambda_1} \mathcal{L} = 0 \longrightarrow R(x_0, x_0, x_1 ; u_1) = 0$$
$$ \partial_{\lambda_{i\neq 1}} \mathcal{L} = 0 \longrightarrow R(x_{i-2}, x_{i-1}, x_i ; u_i) = 0$$


## Derivatives wrt state vectors

$$ \partial_{x_1} J - \lambda_1^T \partial_{x_i} R(x_0, x_0, x_1; u_1) - \lambda_2^T\partial_{x_{i-1}} R(x_0, x_1, x_2; u_2) - \lambda_3^T \partial_{x_{i-2}} R (x_1, x_2, x_3;u_3) = 0 \longrightarrow  \partial_{x_i}^T R(x_0, x_0, x_1; u_1) \lambda_1 = \partial_{x_1}^T J - \partial_{x_{i-1}}^T R(x_0, x_1, x_2; u_2) \lambda_2 - \partial_{x_{i-2}}^T R (x_1, x_2, x_3;u_3)\lambda_3 $$

$$ \partial_{x_i} J - \lambda_i^T \partial_{x_i} R(x_{i-2}, x_{i-1}, x_i; u_i) - \lambda_{i+1}^T\partial_{x_{i-1}} R(x_{i-1}, x_i, x_{i+1}; u_{i+1}) - \lambda_{i+2}^T \partial_{x_{i-2}} R (x_i, x_{i+1}, x_{i+2};u_{i+2}) = 0 \longrightarrow \partial_{x_i}^T R(x_{i-2}, x_{i-1}, x_i; u_i) \lambda_i = \partial_{x_i}^T J  - \partial_{x_{i-1}}^T R(x_{i-1}, x_i, x_{i+1}; u_{i+1})\lambda_{i+1} -  \partial_{x_{i-2}}^T R (x_i, x_{i+1}, x_{i+2};u_{i+2})\lambda_{i+2}$$

$$ \partial_{x_{n-1}} J - \lambda_{n-1}^T \partial_{x_i} R(x_{n-3}, x_{n-2}, x_{n-1}; u_{n-1}) + \lambda_{n}^T\partial_{x_{i-1}} R(x_{n-2}, x_{n-1}, x_{n}; u_{n}) = 0 \longrightarrow \partial_{x_i}^T R(x_{n-3}, x_{n-2}, x_{n-1}; u_{n-1}) \lambda_{n-1}  = \partial_{x_{n-1}}^T J - \partial_{x_{i-1}}^T R(x_{n-2}, x_{n-1}, x_{n}; u_{n})\lambda_{n} $$

$$ \partial_{x_n} J - \lambda_n^T \partial_{x_i} R(x_{n-2}, x_{n-1}, x_n; u_n) = 0 \longrightarrow   \partial_{x_i}^T R(x_{n-2}, x_{n-1}, x_n; u_n) \lambda_n = \partial_{x_n}^T J$$

## Derivative wrt initial condition
$$ \partial_{x_0} J - \lambda_1^T \left(\partial_{x_{i-2}} R(x_0, x_0, x_1; u_0) + \partial_{x_{i-1}} R(x_0, x_0, x_1; u_0)\right) - \lambda_2^T \partial_{x_{i}} R (x_0, x_1, x_2;u_2) = 0$$

## Derivative wrt control parameters
$$ \partial_{u_i} J -\lambda_{i}^T \partial_{u_i} R(x_{i-2}, x_{i-1}, x_i; u_i) = 0 $$

# Derivatives...

$$ R(x_{i-2}, x_{i-1}, x_i; u_i) = 0 \longrightarrow  \partial_{x_i} R \partial_{u_i} x_i + \partial_{u_i} R = 0$$

$$ R(x_{i-1}, x_i, x_{i+1}; u_{i+1}) = 0 \longrightarrow \partial_{x_{i-1}} R  \partial_{u_i} x_i + \partial_{x_i} R \partial_{u_i} x_{i+1} = 0$$

$$ R(x_i, x_{i+1}, x_{i+2}; u_{i+2}) = 0 \longrightarrow \partial_{x_{i-2}} R \partial_{u_i} x_i + \partial_{x_{i-1}} R \partial_{u_i} x_{i+1} + \partial_{x_i} R \partial_{u_i} x_{i+2} = 0$$

$$ R(x_{n-2}, x_{n-1}, x_n; u_n) = 0 \longrightarrow \partial_{x_{i-2}} R \partial_{u_i} x_{n-2} + \partial_{x_{i-1}} R \partial_{u_i} x_{n-1} + \partial_{x_i} R \partial_{u_i} x_n = 0$$

# Finally

$$ \frac{dJ}{du_i} = \partial_{u_i}J + \sum_{k=i}^n \partial_{x_k} J \partial_{u_i} {x_k} = \partial_{u_i} J - \lambda_i^T \partial_{u_i} R$$